<a href="https://colab.research.google.com/github/gadekar-pravin/BeastBytes/blob/main/agents/gemini_data_analytics/intro_gemini_data_analytics_http.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Intro to Gemini Data Analytics

| Author |
| --- |
| [Aditya Verma](https://github.com/vermaAstra) |

# Background and Overview
The **Conversational Analytics API** lets you chat with your BigQuery or Looker data anywhere, including embedded Looker dashboards, Slack and other chat apps, or even your own web applications. Your team members can get answers where they need them, when they need them, in the applications they use every day. You can find the [Colab example with the Python SDK here](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/agents/gemini_data_analytics/intro_gemini_data_analytics_sdk.ipynb). This is a **Pre-GA** product. See [documentation](https://cloud.google.com/gemini/docs/conversational-analytics-api/overview) for more details.

Please provide feedback to conversational-analytics-api-feedback@google.com
<br>
### This notebook will help you
1. Authenticate to Google Cloud
2. Add data
3. Perform agent operations (create, list, get, delete)
4. Manage conversations (create, list, get)
5. Ask questions with your agent


# Get Started

## API Enablement

**Please fill in the billing_project form field with your own Google Cloud project.  The project must have the following APIs enabled:**
-  [cloudaicompanion API](https://console.cloud.google.com/apis/library/cloudaicompanion.googleapis.com)
-  [Gemini Data Analytics API](https://console.cloud.google.com/apis/library/geminidataanalytics.googleapis.com)
-  [BQ API](https://console.cloud.google.com/marketplace/product/google/bigquery.googleapis.com)
-  [Dataform API](https://console.cloud.google.com/apis/library/dataform.googleapis.com)
- [Vertex AI API](https://console.cloud.google.com/apis/library/aiplatform.googleapis.com)

You may pass in any BigQuery project/dataset/table for which you have read permissions.


## Auth and Imports

In [1]:
import json
import json as json_lib

from IPython.display import HTML, display
import altair as alt
from google.colab import auth
import pandas as pd
from pygments import formatters, highlight, lexers
import requests

auth.authenticate_user()

access_token = !gcloud auth application-default print-access-token
headers = {
    "Authorization": f"Bearer {access_token[0]}",
    "Content-Type": "application/json",
}

# Datasource and Billing project


*   Define your billing project within Google Cloud
*   Link your agent to data sources (BigQuery, Looker, Looker Studio)


In [2]:
# @title Billing Project and Prompt (System Instruction)

billing_project = "conversational-analytics-1"  # @param {type:"string"}

location = "global"
api_version = "v1beta"

# provide critical context for your Conversational Analytics Agent here
system_instruction = "Think like an Analyst"  # @param {type:"string"}

In [3]:
# @title BigQuery Datasource

bigquery_data_sources = {
    "bq": {
        "tableReferences": [
            {
                "projectId": "bigquery-public-data",
                "datasetId": "faa",
                "tableId": "us_airports",
            },
            {
                "projectId": "bigquery-public-data",
                "datasetId": "san_francisco",
                "tableId": "street_trees",
            },
            # Add more table references here
        ]
    }
}

# optional example queries (only leveraged for BigQuery datasources currently)
example_queries = [
    {
        "naturalLanguageQuestion": "What is the highest observed positive longitude?",
        "sqlQuery": "SELECT MAX(longitude) FROM airports",
    },
]

In [ ]:
# @title Looker Datasource

# When connecting to the Looker datasource, you can authenticate using either:
# 1. client_id and client_secret
# 2. access_token
looker_credentials = {
    "oauth": {
        "secret": {
            "client_id": "Enter your Client ID Here",
            "client_secret": "Enter your Client Secret Here",
        }
    }
}

# Uncomment this and comment out the above one if you are using access_token for authentication
# looker_credentials = {
#     "oauth": {
#         "token": {
#           "access_token": "your_looker_access_token",
#         }
#     }
# }


# Looker datasource for PUBLIC Instance
looker_data_source = {
    "looker": {
        "explore_references": [
            {
                "looker_instance_uri": "https://my_company.looker.com",
                "lookml_model": "my_model",
                "explore": "my_explore",
            },
        ],
        # NOTE -
        # 1. Do not pass looker credentials during create data agent.
        # 2. Uncomment the below line only when running the method [Stateless Chat using inline context]
        # "credentials": looker_credentials
    }
}


# # Looker datasource for PRIVATE Instance

# looker_data_source = {
#     "looker": {
#       "explore_references": [
#           {
#             "private_looker_instance_info": {
#                 "looker_instance_id": "your_private_looker_instance_id",
#                 "service_directory_name": "your_private_looker_instance_service_directory_name",
#             },
#             "lookml_model": "my_model",
#             "explore": "my_explore",
#           },
#       ],
#       # NOTE -
#       # 1. Do not pass looker credentials during create data agent.
#       # 2. Uncomment the lines below only when running the method [Stateless Chat using inline context]
#       # "credentials": {
#       #     "oauth": {
#       #         "token": {
#       #           "access_token": "your_looker_access_token",
#       #         }
#       #     }
#       # }
#     }
# }

In [ ]:
# @title Looker Studio Datasource

looker_studio_data_source = {
    "studio": {"studio_references": [{"datasource_id": "your_studio_datasource_id"}]}
}

# Environment

In [4]:
# ENVIRONMENT OPTIONS: prod, autopush, staging
environment = "prod"  # @param {type:"string"}

base_url = "https://geminidataanalytics.googleapis.com"

if environment == "autopush":
    base_url = "https://autopush-geminidataanalytics.sandbox.googleapis.com"
elif environment == "staging":
    base_url = "https://staging-geminidataanalytics.sandbox.googleapis.com"
else:
    base_url = "https://geminidataanalytics.googleapis.com"

# Data Agent Services

In [24]:
# @title Create Data Agent

data_agent_id = "data_agent_2"  # @param {type:"string"}

data_agent_url = f"{base_url}/{api_version}/projects/{billing_project}/locations/{location}/dataAgents"

data_agent_payload = {
    "name": f"projects/{billing_project}/locations/{location}/dataAgents/{data_agent_id}",  # Optional
    "description": "This is the description of data_agent.",  # Optional
    "data_analytics_agent": {
        "published_context": {
            "datasource_references": bigquery_data_sources,
            "system_instruction": system_instruction,
            "example_queries": example_queries,  # Optional - Only supported for BigQuery
            "options": {
                "analysis": {
                    # Optional - if wanting to use advanced analysis with python
                    "python": {"enabled": True}
                }
            },
        }
    },
}

params = {"data_agent_id": data_agent_id}  # Optional

data_agent_response = requests.post(
    data_agent_url, params=params, json=data_agent_payload, headers=headers
)

if data_agent_response.status_code == 200:
    print("Data Agent created successfully!")
    print(json.dumps(data_agent_response.json(), indent=2))
else:
    print(f"Error creating Data Agent: {data_agent_response.status_code}")
    print(data_agent_response.text)

Data Agent created successfully!
{
  "name": "projects/conversational-analytics-1/locations/global/operations/operation-1756570418960-63d976bd8573f-ee11fcd4-cc6bdc3a",
  "metadata": {
    "@type": "type.googleapis.com/google.cloud.geminidataanalytics.v1beta.OperationMetadata",
    "createTime": "2025-08-30T16:13:39.087871085Z",
    "target": "projects/conversational-analytics-1/locations/global/dataAgents/data_agent_2",
    "verb": "create",
    "requestedCancellation": false,
    "apiVersion": "v1beta"
  },
  "done": false
}


In [25]:
# @title Get Data Agent

data_agent_id = "data_agent_2"  # @param {type:"string"}

data_agent_url = f"{base_url}/{api_version}/projects/{billing_project}/locations/{location}/dataAgents/{data_agent_id}"

data_agent_response = requests.get(data_agent_url, headers=headers)

if data_agent_response.status_code == 200:
    print("Fetched Data Agent successfully!")
    print(json.dumps(data_agent_response.json(), indent=2))
else:
    print(f"Error: {data_agent_response.status_code}")
    print(data_agent_response.text)

Fetched Data Agent successfully!
{
  "name": "projects/conversational-analytics-1/locations/global/dataAgents/data_agent_2",
  "description": "This is the description of data_agent.",
  "createTime": "2025-08-30T16:13:39.000797765Z",
  "updateTime": "2025-08-30T16:13:40.034281619Z",
  "dataAnalyticsAgent": {
    "publishedContext": {
      "systemInstruction": "Think like an Analyst",
      "options": {
        "analysis": {
          "python": {
            "enabled": true
          }
        }
      },
      "exampleQueries": [
        {
          "naturalLanguageQuestion": "What is the highest observed positive longitude?",
          "sqlQuery": "SELECT MAX(longitude) FROM airports"
        }
      ],
      "datasourceReferences": {
        "bq": {
          "tableReferences": [
            {
              "projectId": "bigquery-public-data",
              "datasetId": "faa",
              "tableId": "us_airports"
            },
            {
              "projectId": "bigquery-pub

In [7]:
# @title List Data Agent

data_agent_url = f"{base_url}/{api_version}/projects/{billing_project}/locations/{location}/dataAgents"

data_agent_response = requests.get(data_agent_url, headers=headers)

if data_agent_response.status_code == 200:
    print("Data Agent Listed successfully!")
    print(json.dumps(data_agent_response.json(), indent=2))
else:
    print(f"Error Listing Data Agent: {data_agent_response.status_code}")
    print(data_agent_response.text)

Data Agent Listed successfully!
{
  "dataAgents": [
    {
      "name": "projects/conversational-analytics-1/locations/global/dataAgents/data_agent_1",
      "description": "This is the description of data_agent.",
      "createTime": "2025-08-30T16:11:13.619899596Z",
      "updateTime": "2025-08-30T16:11:14.753439738Z",
      "dataAnalyticsAgent": {
        "publishedContext": {
          "systemInstruction": "Think like an Analyst",
          "options": {
            "analysis": {
              "python": {
                "enabled": true
              }
            }
          },
          "exampleQueries": [
            {
              "naturalLanguageQuestion": "What is the highest observed positive longitude?",
              "sqlQuery": "SELECT MAX(longitude) FROM airports"
            }
          ],
          "datasourceReferences": {
            "bq": {
              "tableReferences": [
                {
                  "projectId": "bigquery-public-data",
                  "

In [26]:
# @title Update Data Agent

data_agent_id = "data_agent_2"  # @param {type:"string"}

data_agent_url = f"{base_url}/{api_version}/projects/{billing_project}/locations/{location}/dataAgents/{data_agent_id}"

payload = {
    "description": "This is the latest description of data_agent.",
    "data_analytics_agent": {
        "published_context": {
            "datasource_references": bigquery_data_sources,
            "system_instruction": system_instruction,
        }
    },
}

fields = ["description", "data_analytics_agent"]
params = {"updateMask": ",".join(fields)}

data_agent_response = requests.patch(
    data_agent_url, headers=headers, params=params, json=payload
)

if data_agent_response.status_code == 200:
    print("Data Agent updated successfully!")
    print(json.dumps(data_agent_response.json(), indent=2))
else:
    print(f"Error Updating Data Agent: {data_agent_response.status_code}")
    print(data_agent_response.text)

Data Agent updated successfully!
{
  "name": "projects/conversational-analytics-1/locations/global/operations/operation-1756570431920-63d976c9e1985-36f5d964-7a1f674b",
  "metadata": {
    "@type": "type.googleapis.com/google.cloud.geminidataanalytics.v1beta.OperationMetadata",
    "createTime": "2025-08-30T16:13:51.991314021Z",
    "target": "projects/conversational-analytics-1/locations/global/dataAgents/data_agent_2",
    "verb": "update",
    "requestedCancellation": false,
    "apiVersion": "v1beta"
  },
  "done": false
}


In [27]:
# @title [Agent Sharing] Set IAM Policy for Data Agent


""" PLEASE NOTE THIS API CALLS OVERRIDES EXISTING PERMISSION FOR THE RESOURCE.
For preserving existing policy in practise call Get IAM policy to fetch existing policy and pass it along with additional changes
in the call to Set IAM Policy
"""
data_agent_id = "data_agent_2"  # @param {type:"string"}
users = "abc@google.com, wxyz@google.com"  # @param {type:"string"}

data_agent_url = f"{base_url}/{api_version}/projects/{billing_project}/locations/{location}/dataAgents/{data_agent_id}:setIamPolicy"


# Request body
payload = {
    "policy": {
        "bindings": [
            {
                "role": "roles/geminidataanalytics.dataAgentEditor",
                "members": [f"user:{i.strip()}" for i in users.split(",")],
            }
        ]
    }
}

data_agent_response = requests.post(data_agent_url, headers=headers, json=payload)

if data_agent_response.status_code == 200:
    print("IAM Policy set successfully!")
    print(json.dumps(data_agent_response.json(), indent=2))
else:
    print(f"Error setting IAM policy: {data_agent_response.status_code}")
    print(data_agent_response.text)

IAM Policy set successfully!
{
  "version": 1,
  "etag": "BwY9l20sVlA=",
  "bindings": [
    {
      "role": "roles/geminidataanalytics.dataAgentEditor",
      "members": [
        "user:abc@google.com",
        "user:wxyz@google.com"
      ]
    }
  ]
}


In [28]:
# @title [Agent Sharing] Get IAM Policy for Data Agent

data_agent_id = "data_agent_2"  # @param {type:"string"}

data_agent_url = f"{base_url}/{api_version}/projects/{billing_project}/locations/{location}/dataAgents/{data_agent_id}:getIamPolicy"

# Request body
payload = {
    "resource": f"projects/{billing_project}/locations/{location}/dataAgents/{data_agent_id}"
}

data_agent_response = requests.post(data_agent_url, headers=headers, json=payload)

if data_agent_response.status_code == 200:
    print("IAM Policy fetched successfully!")
    print(json.dumps(data_agent_response.json(), indent=2))
else:
    print(f"Error fetching IAM policy: {data_agent_response.status_code}")
    print(data_agent_response.text)

IAM Policy fetched successfully!
{
  "version": 1,
  "etag": "BwY9l20sVlA=",
  "bindings": [
    {
      "role": "roles/geminidataanalytics.dataAgentEditor",
      "members": [
        "user:abc@google.com",
        "user:wxyz@google.com"
      ]
    }
  ]
}


In [11]:
# @title [Soft Delete] Delete Data Agent

data_agent_id = "data_agent_1"  # @param {type:"string"}

data_agent_url = f"{base_url}/{api_version}/projects/{billing_project}/locations/{location}/dataAgents/{data_agent_id}"

data_agent_response = requests.delete(data_agent_url, headers=headers)

if data_agent_response.status_code == 200:
    print("Data Agent deleted successfully!")
    print(json.dumps(data_agent_response.json(), indent=2))
else:
    print(f"Error Deleting Data Agent: {data_agent_response.status_code}")
    print(data_agent_response.text)

Data Agent deleted successfully!
{
  "name": "projects/conversational-analytics-1/locations/global/operations/operation-1756570303947-63d9764fd6498-c2eceb5c-4c1772fc",
  "metadata": {
    "@type": "type.googleapis.com/google.cloud.geminidataanalytics.v1beta.OperationMetadata",
    "createTime": "2025-08-30T16:11:44.026946148Z",
    "target": "projects/conversational-analytics-1/locations/global/dataAgents/data_agent_1",
    "verb": "delete",
    "requestedCancellation": false,
    "apiVersion": "v1beta"
  },
  "done": false
}


# Data Chat Service

In [30]:
# @title Create Conversation

data_agent_id = "data_agent_2"  # @param {type:"string"}
conversation_id = "conversation_2"  # @param {type:"string"}

conversation_url = f"{base_url}/{api_version}/projects/{billing_project}/locations/{location}/conversations"

conversation_payload = {
    "agents": [
        f"projects/{billing_project}/locations/{location}/dataAgents/{data_agent_id}"
    ],
    "name": f"projects/{billing_project}/locations/{location}/conversations/{conversation_id}",
}
params = {"conversation_id": conversation_id}

conversation_response = requests.post(
    conversation_url, headers=headers, params=params, json=conversation_payload
)

if conversation_response.status_code == 200:
    print("Conversation created successfully!")
    print(json.dumps(conversation_response.json(), indent=2))
else:
    print(f"Error creating Conversation: {conversation_response.status_code}")
    print(conversation_response.text)

Conversation created successfully!
{
  "name": "projects/conversational-analytics-1/locations/global/conversations/conversation_2",
  "agents": [
    "projects/conversational-analytics-1/locations/global/dataAgents/data_agent_2"
  ],
  "createTime": "2025-08-30T16:14:30.950368Z",
  "lastUsedTime": "2025-08-30T16:14:31.064050Z"
}


In [31]:
# @title Get Conversation

conversation_id = "conversation_2"  # @param {type:"string"}

conversation_url = f"{base_url}/{api_version}/projects/{billing_project}/locations/{location}/conversations/{conversation_id}"

conversation_response = requests.get(conversation_url, headers=headers)

# Handle the response
if conversation_response.status_code == 200:
    print("Conversation fetched successfully!")
    print(json.dumps(conversation_response.json(), indent=2))
else:
    print(f"Error while fetching conversation: {conversation_response.status_code}")
    print(conversation_response.text)

Conversation fetched successfully!
{
  "name": "projects/conversational-analytics-1/locations/global/conversations/conversation_2",
  "agents": [
    "projects/conversational-analytics-1/locations/global/dataAgents/data_agent_2"
  ],
  "createTime": "2025-08-30T16:14:30.950368Z",
  "lastUsedTime": "2025-08-30T16:14:31.064050Z"
}


In [32]:
# @title List Conversation

conversation_url = f"{base_url}/{api_version}/projects/{billing_project}/locations/{location}/conversations"


conversation_response = requests.get(conversation_url, headers=headers)

# Handle the response
if conversation_response.status_code == 200:
    print("Conversation fetched successfully!")
    print(json.dumps(conversation_response.json(), indent=2))
else:
    print(f"Error while fetching conversation: {conversation_response.status_code}")
    print(conversation_response.text)

Conversation fetched successfully!
{
  "conversations": [
    {
      "name": "projects/conversational-analytics-1/locations/global/conversations/conversation_2",
      "agents": [
        "projects/conversational-analytics-1/locations/global/dataAgents/data_agent_2"
      ],
      "createTime": "2025-08-30T16:14:30.950368Z",
      "lastUsedTime": "2025-08-30T16:14:31.064050Z"
    },
    {
      "name": "projects/conversational-analytics-1/locations/global/conversations/conversation_1",
      "agents": [
        "projects/conversational-analytics-1/locations/global/dataAgents/data_agent_1"
      ],
      "createTime": "2025-08-30T16:12:43.547824Z",
      "lastUsedTime": "2025-08-30T16:12:44.086220Z"
    }
  ]
}


In [21]:
# @title Streaming Chat Messages


def is_json(str):
    try:
        json_object = json_lib.loads(str)
    except ValueError as e:
        return False
    return True


def handle_text_response(resp):
    parts = resp["parts"]
    print("".join(parts))


def get_property(data, field_name, default=""):
    return data[field_name] if field_name in data else default


def display_schema(data):
    fields = data["fields"]
    df = pd.DataFrame(
        {
            "Column": map(lambda field: get_property(field, "name"), fields),
            "Type": map(lambda field: get_property(field, "type"), fields),
            "Description": map(
                lambda field: get_property(field, "description", "-"), fields
            ),
            "Mode": map(lambda field: get_property(field, "mode"), fields),
        }
    )
    display(df)


def display_section_title(text):
    display(HTML(f"<h2>{text}</h2>"))


def format_bq_table_ref(table_ref):
    return "{}.{}.{}".format(
        table_ref["projectId"], table_ref["datasetId"], table_ref["tableId"]
    )


def format_looker_table_ref(table_ref):
    return "lookmlModel: {}, explore: {}".format(
        table_ref["lookmlModel"], table_ref["explore"]
    )


def display_datasource(datasource):
    source_name = ""

    if "studioDatasourceId" in datasource:
        source_name = datasource["studioDatasourceId"]
    elif "lookerExploreReference" in datasource:
        source_name = format_looker_table_ref(datasource["lookerExploreReference"])
    else:
        source_name = format_bq_table_ref(datasource["bigqueryTableReference"])

    print(source_name)
    display_schema(datasource["schema"])


def handle_schema_response(resp):
    if "query" in resp:
        print(resp["query"]["question"])
    elif "result" in resp:
        display_section_title("Schema resolved")
        print("Data sources:")
        for datasource in resp["result"]["datasources"]:
            display_datasource(datasource)


def handle_data_response(resp):
    if "query" in resp:
        query = resp["query"]
        display_section_title("Retrieval query")
        print("Query name: {}".format(query["name"]))
        print("Question: {}".format(query["question"]))
        print("Data sources:")
        for datasource in query["datasources"]:
            display_datasource(datasource)
    elif "generatedSql" in resp:
        display_section_title("SQL generated")
        print(resp["generatedSql"])
    elif "result" in resp:
        display_section_title("Data retrieved")

        fields = map(
            lambda field: get_property(field, "name"),
            resp["result"]["schema"]["fields"],
        )
        dict = {}

        for field in fields:
            dict[field] = map(
                lambda el: get_property(el, field), resp["result"]["data"]
            )

        display(pd.DataFrame(dict))


def handle_chart_response(resp):
    if "query" in resp:
        print(resp["query"]["instructions"])
    elif "result" in resp:
        vegaConfig = resp["result"]["vegaConfig"]
        alt.Chart.from_json(json_lib.dumps(vegaConfig)).display()


def handle_error(resp):
    display_section_title("Error")
    print("Code: {}".format(resp["code"]))
    print("Message: {}".format(resp["message"]))


def get_stream(url, json):
    s = requests.Session()

    acc = ""

    with s.post(url, json=json, headers=headers, stream=True) as resp:
        for line in resp.iter_lines():
            if not line:
                continue

            decoded_line = str(line, encoding="utf-8")

            if decoded_line == "[{":
                acc = "{"
            elif decoded_line == "}]":
                acc += "}"
            elif decoded_line == ",":
                continue
            else:
                acc += decoded_line

            if not is_json(acc):
                continue

            data_json = json_lib.loads(acc)

            if not "systemMessage" in data_json:
                if "error" in data_json:
                    handle_error(data_json["error"])
                continue

            if "text" in data_json["systemMessage"]:
                handle_text_response(data_json["systemMessage"]["text"])
            elif "schema" in data_json["systemMessage"]:
                handle_schema_response(data_json["systemMessage"]["schema"])
            elif "data" in data_json["systemMessage"]:
                handle_data_response(data_json["systemMessage"]["data"])
            elif "chart" in data_json["systemMessage"]:
                handle_chart_response(data_json["systemMessage"]["chart"])
            else:
                colored_json = highlight(
                    acc, lexers.JsonLexer(), formatters.TerminalFormatter()
                )
                print(colored_json)
            print("\n")
            acc = ""

In [33]:
# @title [Stateful] Chat using Conversation
data_agent_id = "data_agent_2"  # @param {type:"string"}
conversation_id = "conversation_2"  # @param {type:"string"}
question = "Make a bar graph for the top 5 states by the total number of airports"  # @param {type:"string"}

chat_url = (
    f"{base_url}/{api_version}/projects/{billing_project}/locations/{location}:chat"
)

# Construct the payload
chat_payload = {
    "parent": f"projects/{billing_project}/locations/global",
    "messages": [{"userMessage": {"text": question}}],
    "conversation_reference": {
        "conversation": f"projects/{billing_project}/locations/{location}/conversations/{conversation_id}",
        "data_agent_context": {
            "data_agent": f"projects/{billing_project}/locations/{location}/dataAgents/{data_agent_id}",
            # "credentials": looker_credentials #Uncomment this if your dataAgent contains looker as dataSource
        },
    },
}

# Call get_stream method to stream the response
get_stream(chat_url, chat_payload)

What are the top 5 states by the total number of airports?




Data sources:
bigquery-public-data.faa.us_airports


,Column,Type,Description,Mode
0,object_id,STRING,-,NULLABLE
1,global_id,STRING,-,NULLABLE
2,faa_identifier,STRING,-,NULLABLE
3,name,STRING,-,NULLABLE
4,latitude,FLOAT,-,NULLABLE
5,longitude,FLOAT,-,NULLABLE
6,airport_geom,GEOGRAPHY,-,NULLABLE
7,elevation,FLOAT,-,NULLABLE
8,icao_id,STRING,-,NULLABLE
9,airport_type,STRING,-,NULLABLE


bigquery-public-data.san_francisco.street_trees


,Column,Type,Description,Mode
0,tree_id,INT64,Unique ID for Tree,REQUIRED
1,legal_status,STRING,Legal staus: Permitted or DPW maintained,NULLABLE
2,species,STRING,Species of tree,NULLABLE
3,address,STRING,Address of Tree,NULLABLE
4,site_order,INT64,Order of tree at address where multiple trees ...,NULLABLE
5,site_info,STRING,Description of location of tree,NULLABLE
6,plant_type,STRING,Landscaping or Tree,NULLABLE
7,care_taker,STRING,Agency or person that is primary caregiver to ...,NULLABLE
8,care_assistant,STRING,Agency or person that is secondary caregiver t...,NULLABLE
9,plant_date,TIMESTAMP,Date tree was planted,NULLABLE


...

SELECT state_abbreviation, COUNT(faa_identifier) AS total_airports
FROM `bigquery-public-data`.`faa`.`us_airports`
GROUP BY state_abbreviation
ORDER BY total_airports DESC
LIMIT 5;






,state_abbreviation,total_airports
0,2033,2033
1,886,886
2,870,870
3,762,762
4,759,759




Generate a bar chart with 'state_abbreviation' on the x-axis and 'total_airports' on the y-axis. The input data do not need to be filtered.




alt.Chart(...)



Here is a bar graph showing the top 5 states by the total number of airports:

```json
{"data": {"values": [{"state_abbreviation": "TX", "total_airports": 2033}, {"state_abbreviation": "FL", "total_airports": 886}, {"state_abbreviation": "CA", "total_airports": 870}, {"state_abbreviation": "AK", "total_airports": 762}, {"state_abbreviation": "PA", "total_airports": 759}]}, "encoding": {"x": {"axis": {"labelOverlap": true}, "field": "state_abbreviation", "sort": {}, "title": "State Abbreviation", "type": "nominal"}, "y": {"axis": {"labelOverlap": true}, "field": "total_airports", "sort": {}, "title": "Total Airports", "type": "quantitative"}}, "mark": {"tooltip": true, "type": "bar"}, "title": "Total Airports by State"}
```




In [34]:
# @title [Stateless] Chat using Data Agent

data_agent_id = "data_agent_2"  # @param {type:"string"}
question = "Make a bar graph for the top 5 states by the total number of airports"  # @param {type:"string"}

chat_url = (
    f"{base_url}/{api_version}/projects/{billing_project}/locations/{location}:chat"
)

# Construct the payload
chat_payload = {
    "parent": f"projects/{billing_project}/locations/global",
    "messages": [{"userMessage": {"text": question}}],
    "data_agent_context": {
        "data_agent": f"projects/{billing_project}/locations/{location}/dataAgents/{data_agent_id}",
        # "credentials": looker_credentials #Uncomment this if your dataAgent contains looker as dataSource
    },
}

# Call get_stream method to stream the response
get_stream(chat_url, chat_payload)

total number of airports by state




Data sources:
bigquery-public-data.faa.us_airports


,Column,Type,Description,Mode
0,object_id,STRING,-,NULLABLE
1,global_id,STRING,-,NULLABLE
2,faa_identifier,STRING,-,NULLABLE
3,name,STRING,-,NULLABLE
4,latitude,FLOAT,-,NULLABLE
5,longitude,FLOAT,-,NULLABLE
6,airport_geom,GEOGRAPHY,-,NULLABLE
7,elevation,FLOAT,-,NULLABLE
8,icao_id,STRING,-,NULLABLE
9,airport_type,STRING,-,NULLABLE


bigquery-public-data.san_francisco.street_trees


,Column,Type,Description,Mode
0,tree_id,INT64,Unique ID for Tree,REQUIRED
1,legal_status,STRING,Legal staus: Permitted or DPW maintained,NULLABLE
2,species,STRING,Species of tree,NULLABLE
3,address,STRING,Address of Tree,NULLABLE
4,site_order,INT64,Order of tree at address where multiple trees ...,NULLABLE
5,site_info,STRING,Description of location of tree,NULLABLE
6,plant_type,STRING,Landscaping or Tree,NULLABLE
7,care_taker,STRING,Agency or person that is primary caregiver to ...,NULLABLE
8,care_assistant,STRING,Agency or person that is secondary caregiver t...,NULLABLE
9,plant_date,TIMESTAMP,Date tree was planted,NULLABLE


...

SELECT state_abbreviation, COUNT(DISTINCT faa_identifier) AS count_of_airports
FROM `bigquery-public-data`.`faa`.`us_airports`
GROUP BY state_abbreviation
ORDER BY COUNT(DISTINCT faa_identifier) DESC
LIMIT 5;






,state_abbreviation,count_of_airports
0,2033,2033
1,886,886
2,870,870
3,762,762
4,759,759




Generate a bar chart with 'state_abbreviation' on the x-axis and 'count_of_airports' on the y-axis. The input data do not need to be filtered.




alt.Chart(...)



The chart displays the top 5 states by the total number of airports. Texas has 2033 airports, Florida has 886, California has 870, Alaska has 762, and Pennsylvania has 759.




In [35]:
# @title [Stateless] Chat using Inline Context

chat_url = f"{base_url}/{api_version}/projects/{billing_project}/locations/global:chat"

question = "Show the schema of datasource"  # @param {type:"string"}

# Construct the payload
chat_payload = {
    "parent": f"projects/{billing_project}/locations/global",
    "messages": [{"userMessage": {"text": question}}],
    "inline_context": {
        "datasource_references": bigquery_data_sources,
        "example_queries": example_queries,  # Optional - Only supported for BigQuery
        "options": {
            "analysis": {
                # Optional - if wanting to use advanced analysis with python
                "python": {"enabled": True}
            }
        },
    },
}

# Call get_stream method to stream the response
get_stream(chat_url, chat_payload)

Show me the schema of the datasources




Data sources:
bigquery-public-data.faa.us_airports


,Column,Type,Description,Mode
0,object_id,STRING,-,NULLABLE
1,global_id,STRING,-,NULLABLE
2,faa_identifier,STRING,-,NULLABLE
3,name,STRING,-,NULLABLE
4,latitude,FLOAT,-,NULLABLE
5,longitude,FLOAT,-,NULLABLE
6,airport_geom,GEOGRAPHY,-,NULLABLE
7,elevation,FLOAT,-,NULLABLE
8,icao_id,STRING,-,NULLABLE
9,airport_type,STRING,-,NULLABLE


bigquery-public-data.san_francisco.street_trees


,Column,Type,Description,Mode
0,tree_id,INT64,Unique ID for Tree,REQUIRED
1,legal_status,STRING,Legal staus: Permitted or DPW maintained,NULLABLE
2,species,STRING,Species of tree,NULLABLE
3,address,STRING,Address of Tree,NULLABLE
4,site_order,INT64,Order of tree at address where multiple trees ...,NULLABLE
5,site_info,STRING,Description of location of tree,NULLABLE
6,plant_type,STRING,Landscaping or Tree,NULLABLE
7,care_taker,STRING,Agency or person that is primary caregiver to ...,NULLABLE
8,care_assistant,STRING,Agency or person that is secondary caregiver t...,NULLABLE
9,plant_date,TIMESTAMP,Date tree was planted,NULLABLE




The available datasources and their schemas are:

**1. bigquery-public-data.faa.us_airports**
*   `object_id`: STRING
*   `global_id`: STRING
*   `faa_identifier`: STRING
*   `name`: STRING
*   `latitude`: FLOAT
*   `longitude`: FLOAT
*   `airport_geom`: GEOGRAPHY
*   `elevation`: FLOAT
*   `icao_id`: STRING
*   `airport_type`: STRING
*   `service_city`: STRING
*   `state_abbreviation`: STRING
*   `country`: STRING
*   `oper_status`: STRING
*   `airport_use`: STRING
*   `iap_exists`: BOOL
*   `dod_hiflip`: BOOL
*   `far_91`: BOOL
*   `far_93`: BOOL
*   `mil_code`: STRING
*   `airspace_analysis`: STRING
*   `us_high`: BOOL
*   `us_low`: BOOL
*   `ak_high`: BOOL
*   `ak_low`: BOOL
*   `us_area`: BOOL
*   `pacific`: BOOL

**2. bigquery-public-data.san_francisco.street_trees**
*   `tree_id`: INT64 (Unique ID for Tree)
*   `legal_status`: STRING (Legal status: Permitted or DPW maintained)
*   `species`: STRING (Species of tree)
*   `address`: STRING (Address of Tree)
*   `site_order`: INT

In [36]:
# @title List Messages

conversation_id = "conversation_2"  # @param {type:"string"}

conversation_url = f"{base_url}/{api_version}/projects/{billing_project}/locations/{location}/conversations/{conversation_id}/messages"

conversation_response = requests.get(conversation_url, headers=headers)

# Handle the response
if conversation_response.status_code == 200:
    print("Conversation fetched successfully!")
    print(json.dumps(conversation_response.json(), indent=2))
else:
    print(f"Error while fetching conversation: {conversation_response.status_code}")
    print(conversation_response.text)

Conversation fetched successfully!
{
  "messages": [
    {
      "messageId": "f6c42a84-87d5-4cd2-bb73-097ab8cc2a22",
      "message": {
        "timestamp": "2025-08-30T16:15:12.257Z",
        "systemMessage": {
          "text": {
            "parts": [
              "Here is a bar graph showing the top 5 states by the total number of airports:\n\n```json\n{\"data\": {\"values\": [{\"state_abbreviation\": \"TX\", \"total_airports\": 2033}, {\"state_abbreviation\": \"FL\", \"total_airports\": 886}, {\"state_abbreviation\": \"CA\", \"total_airports\": 870}, {\"state_abbreviation\": \"AK\", \"total_airports\": 762}, {\"state_abbreviation\": \"PA\", \"total_airports\": 759}]}, \"encoding\": {\"x\": {\"axis\": {\"labelOverlap\": true}, \"field\": \"state_abbreviation\", \"sort\": {}, \"title\": \"State Abbreviation\", \"type\": \"nominal\"}, \"y\": {\"axis\": {\"labelOverlap\": true}, \"field\": \"total_airports\", \"sort\": {}, \"title\": \"Total Airports\", \"type\": \"quantitative\"}},

## Multi-turn Conversation with Data Agent or Inline Context

In [37]:
# @title get_stream function for multi-turn conversation

# NOTE: This methods is same as get_stream() method present in Streaming Chat Messages section.
# The only difference is that - here we are storing the response in an array "conversation_messages" to save the conversation.


def get_stream_multi_turn(url, json, conversation_messages):
    s = requests.Session()

    acc = ""

    with s.post(url, json=json, headers=headers, stream=True) as resp:
        for line in resp.iter_lines():
            if not line:
                continue

            decoded_line = str(line, encoding="utf-8")

            if decoded_line == "[{":
                acc = "{"
            elif decoded_line == "}]":
                acc += "}"
            elif decoded_line == ",":
                continue
            else:
                acc += decoded_line

            if not is_json(acc):
                continue

            data_json = json_lib.loads(acc)
            # Store the response to be used in next iteration.
            conversation_messages.append(data_json)

            if not "systemMessage" in data_json:
                if "error" in data_json:
                    handle_error(data_json["error"])
                continue

            if "text" in data_json["systemMessage"]:
                handle_text_response(data_json["systemMessage"]["text"])
            elif "schema" in data_json["systemMessage"]:
                handle_schema_response(data_json["systemMessage"]["schema"])
            elif "data" in data_json["systemMessage"]:
                handle_data_response(data_json["systemMessage"]["data"])
            elif "chart" in data_json["systemMessage"]:
                handle_chart_response(data_json["systemMessage"]["chart"])
            else:
                colored_json = highlight(
                    acc, lexers.JsonLexer(), formatters.TerminalFormatter()
                )
                print(colored_json)
            print("\n")
            acc = ""

In [38]:
# @title Multi-turn Conversation helper function
chat_url = f"{base_url}/{api_version}/projects/{billing_project}/locations/global:chat"

# Re-used across requests to track previous turns
conversation_messages = []

data_agent_id = "data_agent_2"  # @param {type:"string"}

# Helper function for calling the API


def multi_turn_conversation(msg):

    userMessage = {"userMessage": {"text": msg}}

    # Send multi-turn request by including previous turns, plus new message
    conversation_messages.append(userMessage)

    # Construct the payload
    chat_payload = {
        "parent": f"projects/{billing_project}/locations/global",
        "messages": conversation_messages,
        "data_agent_context": {
            "data_agent": f"projects/{billing_project}/locations/{location}/dataAgents/{data_agent_id}",
            # "credentials": looker_credentials # Uncomment this if your dataAgent contains looker as dataSource
        },
        # uncomment the lines below when using chat with inline_context
        # "inline_context": {
        #     "datasource_references": bigquery_data_sources,
        # }
    }

    # Call get_stream_multi_turn method to stream the response
    get_stream_multi_turn(chat_url, chat_payload, conversation_messages)

In [39]:
# Send first-turn request
multi_turn_conversation("List of the top 5 states by the total number of airports")

List of the top 5 states by the total number of airports




Data sources:
bigquery-public-data.faa.us_airports


,Column,Type,Description,Mode
0,object_id,STRING,-,NULLABLE
1,global_id,STRING,-,NULLABLE
2,faa_identifier,STRING,-,NULLABLE
3,name,STRING,-,NULLABLE
4,latitude,FLOAT,-,NULLABLE
5,longitude,FLOAT,-,NULLABLE
6,airport_geom,GEOGRAPHY,-,NULLABLE
7,elevation,FLOAT,-,NULLABLE
8,icao_id,STRING,-,NULLABLE
9,airport_type,STRING,-,NULLABLE


bigquery-public-data.san_francisco.street_trees


,Column,Type,Description,Mode
0,tree_id,INT64,Unique ID for Tree,REQUIRED
1,legal_status,STRING,Legal staus: Permitted or DPW maintained,NULLABLE
2,species,STRING,Species of tree,NULLABLE
3,address,STRING,Address of Tree,NULLABLE
4,site_order,INT64,Order of tree at address where multiple trees ...,NULLABLE
5,site_info,STRING,Description of location of tree,NULLABLE
6,plant_type,STRING,Landscaping or Tree,NULLABLE
7,care_taker,STRING,Agency or person that is primary caregiver to ...,NULLABLE
8,care_assistant,STRING,Agency or person that is secondary caregiver t...,NULLABLE
9,plant_date,TIMESTAMP,Date tree was planted,NULLABLE


...

SELECT state_abbreviation, COUNT(DISTINCT faa_identifier) AS count_of_airports
FROM `bigquery-public-data`.`faa`.`us_airports`
GROUP BY state_abbreviation
ORDER BY COUNT(DISTINCT faa_identifier) DESC
LIMIT 5;






,state_abbreviation,count_of_airports
0,2033,2033
1,886,886
2,870,870
3,762,762
4,759,759




Generate a bar chart with state_abbreviation on the x-axis and count_of_airports on the y-axis. The input data do not need to be filtered.




alt.Chart(...)



The top 5 states by the total number of airports are: TX with 2033 airports, FL with 886 airports, CA with 870 airports, AK with 762 airports, and PA with 759 airports. A bar chart visualizing this data has been generated.




In [40]:
# Send follow-up-turn request
multi_turn_conversation("Can you please make it a pie chart?")

Generate a pie chart showing the distribution of airports by state. The input data do not need to be filtered.




alt.Chart(...)



A pie chart showing the distribution of airports by state has been generated.


